Goal: To convert all the .srt to .txt for all the english files

Before even converting the files, I had to run this bash script 
for f in *\ *; do mv -- "$f" "${f// /_}"; done
to clean up the spaces in the data and had to manually change the few en.srt files that was not formatted correctly. Note if this project were to be continued, to use the dataset_epfl that I have here

Note to self: clean up POCS,Ville,Safewater folder - change to en.srt


Here I am creating a new folder for all the english subtitles 

In [1]:
import os
import shutil

In [2]:
path = "/Users/shrutigoli/EPFL_MOOC_NLP_Research/updated_dataset_epfl"
try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
else:  
    print ("Successfully created the directory %s " % path)  

Creation of the directory /Users/shrutigoli/EPFL_MOOC_NLP_Research/updated_dataset_epfl failed


Below I am parsing through all the directories/subdirectories/files and filtering only the files with english subtitles

In [3]:
fileslist = []
for root, dirs, files in os.walk("."):
    for d in dirs:
        fileslist.append(os.path.relpath(os.path.join(root, d), "."))
    for f in files:
        fileslist.append(os.path.relpath(os.path.join(root, f), "."))
    #print(root)
    if root.endswith("en.srt") or root.endswith("ENGL.srt") or root.endswith("EN.srt") :
        !python srt_to_text.py $root cp1252

I attempted to convert the english .srt files to .txt files using the srt_to_text.py converter I found online (details attached in that .py file). I noticed not all of the files converted, the ones listed below with comments below. I had to go through the folders and convert these english .srt files by running the below commands

In [4]:
#astro, safewater, BIOIMG, Brain, Electronique_I/Annexe, Electrotechnique/Semestre01/Final/,
#Electrotechnique/Semestre01/Transifex/ENGL/ , same for the semester2 <--
#Emergency/First_batch (en_Goodman.srt)
#Espace/ENGL/Session1 <-- all Sessions1-9
#Fluids/Teaser <-- all folders in here 
#Innov/WEEK_01/ <-- all folders 1-6
#Market/Week01 
#MEMs/W01/ <-- all of them 1-6
#/OOP_CPP/ENGL/W07/ <-- all of them 1-7
#/OOP_JAVA/ENGL/W07/ <-- all of them 1-7
#POCS/W11/ <-- All of them
#Risk/W07 <-- all of them
#Sanitation/W05/ <-- all of them
#Space/W07/ <-- all of them
#Ville/W12/ <-- all of them
#/WHO/W05/ <-- all of them 1-5

a = '/Users/shrutigoli/EPFL_MOOC_NLP_Research/dataset_epfl/WHO/W05/'
for filename in os.listdir(a):
    b = a + filename
    if b.endswith("en.srt") or b.endswith("ENGL.srt") or b.endswith("EN.srt") :
        !python srt_to_text.py $b cp1252

Here I have filtered out the .txt file names and paths into a list fileslist2 to identify the files we will be using for the rest of the analysis.

In [5]:
fileslist2 = [k for k in fileslist if '.txt' in k]
dest_dir = '/Users/shrutigoli/EPFL_MOOC_NLP_Research/updated_dataset_epfl'
for filename in fileslist2:
    shutil.copy(filename, dest_dir)  

SameFileError: 'updated_dataset_epfl/13-Coupon_bonds_and_interest_rate_swaps-en.txt' and '/Users/shrutigoli/EPFL_MOOC_NLP_Research/updated_dataset_epfl/13-Coupon_bonds_and_interest_rate_swaps-en.txt' are the same file

Out of the fileslist that appends every folder/file and saves this into the list, fileslist2 filters out the 3794 files we will be conducting the NLP analysis on.

In [6]:
print(len(fileslist))
print(len(fileslist2))

18218
3794


In [ ]:
#directory = '/Users/shrutigoli/Masters_Research/updated_dataset_epfl'
#print(os.listdir(directory))

Now that all the files are converted into txt, we have to remove stopwords/lemmatize and clean up

In [ ]:
print (fileslist2[1])

In [7]:
import pandas as pd
import re
import string
import nltk 
ps = nltk.PorterStemmer()

In [8]:
#Creating a dataframe to extract all the important words for the relevant .txt file
df = pd.DataFrame.from_dict(fileslist2)
df.columns = ['file_path']
df.head()
#print(df.shape[0]) #3794
print(df['file_path'][77])

dataset_epfl/Neuron/2.5.2_Understanding_neuronal_morphologies_using_NeuroM_en.txt


In [9]:

stopwords = nltk.corpus.stopwords.words('english')
df_key = pd.DataFrame(columns=['key_words'])

def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

i = 75
count_row = df.shape[0]
for i in range(76):
    txt_path = df['file_path'][i]
    data=pd.read_csv(txt_path, delimiter="\t")

    data.columns = ['body_text']
    data['body_text_nostop'] = data['body_text'].apply(lambda x: clean_text(x.lower()))
    
    #running a for loop to combine all the no stop words into one list 
    mergedlist = []
    for j in range(data.shape[0]): 
        mergedlist = list(set(data['body_text_nostop'][j] + mergedlist ))

        mergedlist = [x.strip('') for x in mergedlist] #supposed to take out spaces but not working 

    df_key.loc[j, 'key_words'] = mergedlist 

#df_key.head(60)

In [10]:
count_row = data.shape[0]
print(count_row)

mergedlist = []
for i in range(count_row): 
    mergedlist = list(set(data['body_text_nostop'][i] + mergedlist ))

119


In [11]:
dfk2 = pd.DataFrame(columns=['A'])
mergedlist2 = mergedlist
mergedlist2.append('cool')
dfk2.at[1, 'A'] = mergedlist
dfk2.at[2, 'A'] = mergedlist2



#ml3 = [mergedlist, mergedlist2]
#print(ml3)
#print(mergedlist)
#dfk = pd.DataFrame(ml3)
dfk2.head()

,A
1,"[brain, measurements, lectures, test, simulate..."
2,"[brain, measurements, lectures, test, simulate..."
